In [ ]:
!pip install datasets scikit-learn sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm

import nltk
from nltk.stem import PorterStemmer
import string
import re
import random

import torch

%matplotlib inline

In [ ]:
SEED=42
def set_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Research UPV/Development Notebooks/Data/data.xlsx')
data.describe()

,Title,label,text,URL
count,7037,7037,7037,7037
unique,6835,43,6835,6835
top,Prosolvable group,Semantics,"In mathematics, more precisely in algebra, a p...",https://en.wikipedia.org/wiki/Prosolvable_group
freq,3,285,3,3


In [ ]:
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

import nltk

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

def process_text(text):
    """Applies a text processing pipeline including lowercasing, punctuation removal, stop word removal, stemming, and lemmatization."""

    text = text.lower()

    table = str.maketrans("", "", string.punctuation)
    text = text.translate(table)

    words = nltk.word_tokenize(text)

    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]

    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    words_stemmed_lemmatized = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words]

    processed_text = " ".join(words_stemmed_lemmatized)

    return processed_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data_text = data['text'].dropna()

data_text = data_text.apply(process_text)

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')
tokenizer = model.tokenizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def chunk_text(text, min_length=1, max_length=512, stride=256):
    """
    Split text into overlapping chunks with CLS and SEP tokens,
    while considering minimum and maximum length.
    """

    tokenized_text = tokenizer.tokenize(text)

    chunks = []
    for i in range(0, len(tokenized_text), stride):
        chunk = tokenized_text[i:i + max_length - 2]
        chunk_with_special_tokens = [tokenizer.cls_token] + chunk + [tokenizer.sep_token]
        chunks.append(chunk_with_special_tokens)

    if len(chunks) > 1:
        chunks = [chunk for chunk in chunks[:-1] if len(chunk) >= min_length + 2] + [chunks[-1]]
    return chunks

def process_article(article):
    # Step 1: Chunk the text
    chunks = chunk_text(article)

    # Step 2: Encode each chunk
    chunk_embeddings = [model.encode(' '.join(chunk), convert_to_tensor=True) for chunk in chunks]

    # Step 3: Stack the embeddings
    # aggregated_embedding = torch.cat(chunk_embeddings, dim=0)
    aggregated_embedding = torch.mean(torch.stack(chunk_embeddings), dim=0)

    # Choose one option based on your need
    return aggregated_embedding.cpu().numpy()  # or averaged_embedding.cpu().numpy()

In [ ]:
embeddings = np.array([process_article(article) for article in tqdm(data_text)])
embeddings.shape, embeddings[0]

100%|██████████| 7037/7037 [13:52<00:00,  8.45it/s]


((7037, 768),
 array([-3.13669490e-03, -2.01363396e-02, -1.24848504e-02,  2.67082229e-02,
        -3.68854702e-02,  1.06865531e-02, -4.29157680e-03, -2.13464964e-02,
         5.70805483e-02, -1.25770206e-02,  3.57664898e-02,  1.14156576e-02,
         4.04701456e-02,  3.15491706e-02,  2.75388714e-02, -6.08727708e-02,
         9.00685787e-03,  1.48336403e-02,  7.90690817e-03, -8.52484535e-03,
         6.38498319e-03,  6.36065286e-03, -4.57317680e-02, -1.17610861e-03,
         3.30552906e-02,  7.83252239e-04, -2.39277650e-02,  1.29562421e-02,
         2.50167791e-02, -2.05592792e-02,  3.36344936e-03,  3.53799723e-02,
        -1.11263758e-02, -2.93681435e-02,  2.78012216e-04, -5.04210629e-02,
        -5.98100992e-03, -3.56575809e-02, -8.47586710e-03,  1.53313139e-02,
         3.37493010e-02,  8.40951279e-02, -8.88924208e-03,  1.63691267e-02,
        -4.94647864e-03, -1.82314608e-02,  2.35443991e-02, -1.65386107e-02,
        -1.23474945e-03,  5.26480898e-02,  2.70487480e-02, -2.47238334e-02

In [ ]:
embeddings_path='/content/drive/MyDrive/Research UPV/Development Notebooks/Data/Embeddings/embeddings.npy'

In [ ]:
np.save(embeddings_path, embeddings)

In [ ]:
embeddings = np.load(embeddings_path)

In [ ]:
from sklearn.manifold import TSNE

X_embedded_2d = TSNE(n_components=2, random_state=SEED).fit_transform(embeddings)
X_embedded_3d = TSNE(n_components=3, random_state=SEED).fit_transform(embeddings)

In [ ]:
!pip install plotly cufflinks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import plotly.express as px

tsne_df = pd.DataFrame(X_embedded_2d, columns=['TSNE1', 'TSNE2'])
tsne_df['label'] = data['label']
tsne_df['Title'] = data['Title']

# Create a 2D scatter plot
fig = px.scatter(tsne_df, x='TSNE1', y='TSNE2', color='label', hover_data=['Title'])
fig.update_layout(title='2D t-SNE of Articles', xaxis_title='TSNE Dimension 1', yaxis_title='TSNE Dimension 2')
fig.write_html("/content/drive/MyDrive/Research UPV/Development Notebooks/Plots/data_plot.html")
fig.show()

In [ ]:
tsne_df_3d = pd.DataFrame(X_embedded_3d, columns=['TSNE1', 'TSNE2', 'TSNE3'])
tsne_df_3d['label'] = data['label']
tsne_df_3d['Title'] = data['Title']

# Create a 3D scatter plot
fig_3d = px.scatter_3d(tsne_df_3d, x='TSNE1', y='TSNE2', z='TSNE3', color='label', hover_data=['Title'])
fig_3d.update_layout(title='3D t-SNE of Articles', margin=dict(l=0, r=0, b=0, t=0))
fig.write_html("/content/drive/MyDrive/Research UPV/Development Notebooks/Plots/ST_Wiki_3d.html")
fig_3d.show()

In [ ]:
!pip install hnswlib

In [ ]:
import hnswlib
import numpy as np
import pandas as pd

dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)

index.init_index(max_elements=len(embeddings), ef_construction=200, M=16)
index.add_items(embeddings)

index.set_ef(50)

def find_best_course(data, q_input):
    if data is not None and q_input is not None:
        # Encode the query input

        text = process_text(q_input)
        query = model.encode([text])[0]

        # Find the 5 nearest neighbors
        ids, distances = index.knn_query(query, k=5)

        best_fits = []
        for i, dist in zip(ids[0], distances[0]):
            best_fits.append({
                'Title': data['Title'].values[i],
                'Label': data['label'].values[i],
                'URL': data['URL'].values[i],
                'distance': dist
            })

        return pd.DataFrame(best_fits)


In [ ]:
# Semantics article
query1 = '''
In linguistics, sound symbolism is the perceptual similarity between speech sounds and concept meanings. It is a form of linguistic iconicity. For example, the English word ding may sound similar to the actual sound of a bell. Linguistic sound may be perceived as similar to not only sounds, but also to other sensory properties, such as size, vision, touch, or smell, or abstract domains, such as emotion or value judgment. Such correspondence between linguistic sound and meaning may significantly affect the form of spoken languages. == In Cratylus, Plato has Socrates commenting on the origins and correctness of various names and words. When Hermogenes asks if he can provide another hypothesis on how signs come into being (his own is simply 'convention'), Socrates initially suggests that they fit their referents in virtue of the sounds they are made of: Now the letter rho, as I was saying, appeared to the imposer of names an excellent instrument for the expression of motion; and he frequently uses the letter for this purpose: for example, in the actual words rein and roe he represents motion by rho; also in the words tromos (trembling), trachus (rugged); and again, in words such as krouein (strike), thrauein (crush), ereikein (bruise), thruptein (break), kermatixein (crumble), rumbein (whirl): of all these sorts of movements he generally finds an expression in the letter R, because, as I imagine, he had observed that the tongue was most agitated and least at rest in the pronunciation of this letter, which he therefore used in order to express motion However, faced by an overwhelming number of counterexamples given by Hermogenes, Socrates has to admit that "my first notions of original names are truly wild and ridiculous". == The Upanishads and Vyākaraṇa contain a lot of material about sound symbolism, for instance: The mute consonants represent the earth, the sibilants the sky, the vowels heaven. The mute consonants represent fire, the sibilants air, the vowels the sun… The mute consonants represent the eye, the sibilants the ear, the vowels the mind. The concept of Sphota and Nirukta is also based on this. == Kūkai, the founder of Shingon, wrote his Sound, word, reality in the 9th century which relates all sounds to the voice of the Dharmakaya Buddha. == The idea of phonosemantics was sporadically discussed during the Middle Ages and the Renaissance. In 1690, Locke wrote against the idea in An Essay Concerning Human Understanding. His argument was that if there were any connection between sounds and ideas, then we would all be speaking the same language, but this is an over-generalisation. Leibniz's book New Essays on Human Understanding published in 1765 contains a point by point critique of Locke's essay. Leibniz picks up on the generalization used by Locke and adopts a less rigid approach: clearly there is no perfect correspondence between words and things, but neither is the relationship completely arbitrary, although he seems vague about what that relationship might be. == Modern linguistics has been seen as opposing sound symbolism, beginning with Ferdinand de Saussure (1857–1913), considered the founder of modern 'scientific' linguistics. Central to what de Saussure says about words are two related statements: First, he says that "the sign is arbitrary". He considers the words that we use to indicate things and concepts could be any words – they are essentially just a consensus agreed upon by the speakers of a language and have no discernible pattern or relationship to the thing. Second, he says that, because words are arbitrary, they have meaning only in relation to other words. A dog is a dog because it is not a cat or a mouse or a horse, etc. These ideas have permeated the study of words since the 19th century. == Onomatopoeia refers to words that imitate sounds. Some examples in English are bow-wow or meow, each representing the sound of a dog or a cat. == An ideophone is "a member of an open lexical class of marked words that depict sensory imagery". Unlike onomatopoeia, an ideophone refers to words that depict any sensory domain, such as vision or touch. Examples are Korean mallang-mallang 말랑말랑 'soft' and Japanese kira-kira キラキラ 'shiny'. Ideophones are heavily present in many African and East/Southeast Asian languages, such as Japanese, Thai, and Xhosa. Their form is very often reduplicated, but not necessarily so. == A phonaestheme is a sub-morphemic sequence of sounds that are associated to a certain range of meanings. A well-known example is English gl-, which is present in many words related to light or vision, such as gleam, glow, or glare. Since it is submorphemic, gl- itself is not a morpheme, and it does not form compounds with other morphemes: -eam, -ow, and -are have no meaning of their own. Phonaesthemes, however, are not necessarily iconic, as they may be language-specific and may not iconically resemble the meaning they are associated to. == Blasi et al. (2016), Joo (2020), and Johansson et al. (2020) demonstrated that in the languages around the world, certain concepts in the basic vocabulary (such as the Swadesh list or the Leipzig-Jakarta list) tend to be represented by words containing certain sounds. Below are some of the phonosemantic associations confirmed by the three studies: == High front vowels, such as /i/, are known to be perceptually associated to small size, whereas low and/or back vowels, such as /u/ or /a/, are usually associated with big size. This phenomenon is known as magnitude symbolism. Sapir (1929) showed that, when asked which of the two tables, named mil and mal, is bigger than the other, many people choose mal to be bigger than mil. This phenomenon is not only observable in pseudowords, but present throughout English vocabulary as well. == In many languages, the proximal demonstrative pronoun ('this') tends to have high front vowels (such as /i/), whereas the distal demonstrative pronoun ('that') tends to have low and/or back vowels (such as /u/). Examples include: English this and that, French ceci and cela, and Indonesian ini and itu. == First person pronouns (me) and second person pronouns (you) tend to contain a nasal sound. Joo (2020) suggests that this may be related to the infant's tendency of using the nasal sound to seek the attention of the caretaker. == Köhler (1929, 1947, 1970) introduced what is known as the Takete-Maluma phenomenon. When presented two shapes, one being curvy and another being spiky, and asked which one is called Takete and which one is called Maluma, participants are more likely to associate the name Takete to the spiky shape and the name Maluma to the curvy shape. Following Ramachandran and Hubbard (2001), this phenomenon is now more commonly known as the Bouba/kiki effect, and has been demonstrated to be valid across different cultures and languages. == Bilabial consonants have been demonstrated to be linked to the perception of softness, arguably due to the soft texture of human lips.The trilled R is frequent in words for 'rough' while infrequent in words for 'smooth'. Sound symbolism is used in commerce for the names of products and even companies themselves. For example, a car company may be interested in how to name their car to make it sound faster or stronger. Furthermore, sound symbolism can be used to create a meaningful relationship between a company's brand name and the brand mark itself. Sound symbolism can relate to the color, shade, shape, and size of the brand mark.
'''

# Algebra article => The Hundred Fowls Problem
query2 = '''
The Hundred Fowls Problem is a problem first discussed in the fifth century CE Chinese mathematics text Zhang Qiujian suanjing (The Mathematical Classic of Zhang Qiujian), a book of mathematical problems written by Zhang Qiujian. It is one of the best known examples of indeterminate problems in the early history of mathematics. The problem appears as the final problem in Zhang Qiujian suanjing (Problem 38 in Chapter 3). However, the problem and its variants have appeared in the medieval mathematical literature of India, Europe and the Arab world.The name "Hundred Fowls Problem" is due to the Belgian historian Louis van Hee. The Hundred Fowls Problem as presented in Zhang Qiujian suanjing can be translated as follows: "Now one cock is worth 5 qian, one hen 3 qian and 3 chicks 1 qian. It is required to buy 100 fowls with 100 qian. In each case, find the number of cocks, hens and chicks bought." Let x be the number of cocks, y be the number of hens, and z be the number of chicks, then the problem is to find x, y and z satisfying the following equations: x + y +z = 100 5x + 3y + z/3 = 100Obviously, only non-negative integer values are acceptable. Expressing y and z in terms of x we get y = 25 − (7/4)x z = 75 + (3/4)xSince x, y and z all must be integers, the expression for y suggests that x must be a multiple of 4. Hence the general solution of the system of equations can be expressed using an integer parameter t as follows: x = 4t y = 25 − 7t z = 75 + 3tSince y should be a non-negative integer, the only possible values of t are 0, 1, 2 and 3. So the complete set of solutions is given by (x,y,z) = (0,25,75), (4,18,78), (8,11,81), (12,4,84).of which the last three have been given in Zhang Qiujian suanjing. However, no general method for solving such problems has been indicated, leading to a suspicion of whether the solutions have been obtained by trial and error.The Hundred Fowls Problem found in Zhang Qiujian suanjing is a special case of the general problem of finding integer solutions of the following system of equations: x + y + z = d ax + by + cz = dAny problem of this type is sometime referred to as "Hundred Fowls problem". Some variants of the Hundred Fowls Problem have appeared in the mathematical literature of several cultures. In the following we present a few sample problems discussed in these cultures. == Mahavira's Ganita-sara-sangraha contains the following problem: Pigeons are sold at the rate of 5 for 3, sarasa-birds at the rate of 7 for 5, swans at the rate of 9 for 7, and peacocks at the rate of 3 for 9 (panas). A certain man was told to bring 100 birds for 100 panas. What does he give for each of the various kinds of birds he buys?The Bakshali manuscript gives the problem of solving the following equations: x + y + z = 20 3x + (3/2)y + (1/2)z = 20 == The English mathematician Alcuin of York (8th century, c.735-19 May 804 AD) has stated seven problems similar to the Hundred Fowls Problem in his Propositiones ad acuendos iuvenes. Here is a typical problem: If 100 bushels of corn be distributed among 100 people such that each man gets 3 bushels, each woman 2 bushels and each child half a bushel, then how many men, women and children were there? == Abu Kamil (850 - 930 CE) considered non-negative integer solutions of the following equations: x + y + z = 100 3x + (/20)y+ (1/3)z = 100.
'''

most_similar_query1 = find_best_course(data, query1)
most_similar_query2 = find_best_course(data, query2)

In [ ]:
most_similar_query1

,Title,Label,URL,distance
0,Sound symbolism,Semantics,https://en.wikipedia.org/wiki/Sound_symbolism,0.253273
1,Statistical semantics,Semantics,https://en.wikipedia.org/wiki/Statistical_sema...,0.315718
2,Structural semantics,Semantics,https://en.wikipedia.org/wiki/Structural_seman...,0.319026
3,Sociohistorical linguistics,Sociolinguistics,https://en.wikipedia.org/wiki/Sociohistorical_...,0.321986
4,Onomasiology,Semantics,https://en.wikipedia.org/wiki/Onomasiology,0.328187


In [ ]:
most_similar_query2

,Title,Label,URL,distance
0,Hundred Fowls Problem,Algebra,https://en.wikipedia.org/wiki/Hundred_Fowls_Pr...,0.358267
1,Hundred Fowls Problem,Number theory,https://en.wikipedia.org/wiki/Hundred_Fowls_Pr...,0.358267
2,Pigeonhole principle,Combinatorics,https://en.wikipedia.org/wiki/Pigeonhole_princ...,0.459702
3,Number theory,Number theory,https://en.wikipedia.org/wiki/Number_theory,0.487696
4,Wild number,Number theory,https://en.wikipedia.org/wiki/Wild_number,0.497118
